In [1]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from deepmatch.models import YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss,NegativeSampler
warnings.filterwarnings('ignore')
from data_reading import get_all_click_sample, get_all_click_df, get_item_info_df, get_item_emb_dict, trn_val_split, get_trn_val_tst_data, get_recall_list, get_embedding, get_article_info_df
from config import get_user_item_time, get_item_user_time_dict, get_hist_and_last_click, get_item_info_dict, get_user_hist_item_info_dict, get_item_topk_click
from metrics_recall import metrics_recall
from sim_matrix import itemcf_sim, get_user_activate_degree_dict, embedding_sim
from sim_matrix import usercf_sim
from sim_matrix import embedding_sim
from recall import item_based_recommend, user_based_recommend, combine_recall_results
from cold_start_items import get_click_article_ids_set, cold_start_items
from config import get_rank_label_df, get_user_recall_item_label_df, make_tuple_func, get_hist_and_last_click, reduce_mem
from neg_sample import recall_dict_2_df, neg_sample_recall_data
from feature_engineering import create_feature, active_level, hot_level, device_fea, user_time_hob_fea, user_cat_hob_fea
from sorting_with_model_ensemble import submit, norm_sim, get_din_feats_columns, get_ensemble_predict_topk
import lightgbm as lgb
from datetime import datetime
from deepctr.models import DIN
import tensorflow as tf
import time

2024-11-02 13:21:44.485335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# 定义数据路径
data_path='/Users/linjiaxi/Desktop/RecommendationSystem/Competition/Alibaba - News Recommendation Competition/data/'
save_path='/Users/linjiaxi/Desktop/RecommendationSystem/Competition/Alibaba - News Recommendation Competition/tmp/'
metric_recall=True

In [6]:
# 定义归一化
max_min_scaler=lambda x: (x-np.min(x))/(np.max(x)-np.min(x))

In [7]:
# 定义数据
# 不适用debug模式 并且获取所有数据

# 用户的点击记录
all_click_df=get_all_click_df(data_path,offline=False)
# 文章的基本属性
item_info_df=get_item_info_df(data_path)
# 文章的embedding字典
item_emb_dict=get_item_emb_dict(data_path)

In [5]:
all_click_df.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [6]:
item_info_df.head()

,click_article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162


In [7]:
num_items=2
for key, value in list(item_emb_dict.items())[:num_items]:
    print(f"{key}:{value}")

0:[-0.02118425 -0.12580893 -0.01813001  0.00668391  0.10909397  0.11846624
 -0.04404838 -0.07354293 -0.06579412  0.02170995  0.05630901  0.04666488
  0.11492702 -0.06951095  0.08220764  0.03534407 -0.10814503 -0.09250724
 -0.08225472 -0.02008969 -0.08756393  0.00569023  0.02347829  0.00616275
  0.07813909 -0.02409734  0.02564285 -0.06146177 -0.04006071  0.04641773
  0.03656221  0.07079111 -0.04878796  0.06438719 -0.01364673  0.01566297
  0.01740611 -0.08162891 -0.0595786   0.04555704 -0.00811461 -0.09602179
 -0.05048424 -0.12364366  0.00806219  0.06342559  0.038073   -0.08184084
 -0.00657207  0.05539924 -0.03188176  0.08788847 -0.06689828 -0.06069421
  0.00577     0.03791584  0.05912034 -0.03743939  0.12048548  0.09241205
  0.11193531 -0.08243855  0.04701659  0.0512825   0.08581513  0.01362305
  0.10491944 -0.01347765 -0.02687927  0.04976083 -0.01186304 -0.03300955
  0.04284385  0.0112872  -0.0237316   0.00624597 -0.02099457 -0.08150581
  0.03947292 -0.10468995  0.06278647  0.05387553 

In [5]:
# 获取文章的属性信息，保存成字典的形式方便查询
item_type_dict, item_words_dict, item_created_time_dict=get_item_info_dict(item_info_df)

In [9]:
num_items=2
for key, value in list(item_type_dict.items())[:num_items]:
    print(f"{key}:{value}")

0:0
1:1


In [10]:
num_items=2
for key,value in list(item_created_time_dict.items())[:num_items]:
    print(f"{key}:{value}")

0:0.9784319658749242
1:0.6802953033702287


In [11]:
num_items=2
for key,value in list(item_words_dict.items())[:num_items]:
    print(f"{key}:{value}")

0:168
1:189


In [6]:
# 定义一个多路召回的字典，将各路召回的结果都保存在这个字典中
user_multi_recall_dict={'itemcf_sim_itemcf_recall':{},
                        'embedding_sim_item_recall':{},
                        'cold_start_recall':{}}

In [7]:
user_item_time_dict=get_user_item_time(all_click_df)

In [14]:
# 基于embedding的相似性矩阵
item_emb_df = pd.read_csv(data_path + '/articles_emb.csv')
emb_i2i_sim = embedding_sim(all_click_df, item_emb_df, save_path, topk=10) # topk可以自行设置

364047it [00:15, 22955.81it/s]


In [15]:
# 基于物品的协同过滤的相似性矩阵
i2i_sim = itemcf_sim(all_click_df, item_created_time_dict)

100%|██████████| 250000/250000 [07:06<00:00, 586.39it/s] 


In [16]:
# 基于用户的协同过滤的相似性矩阵（跑得太慢了，所以先不跑了）
user_activate_degree_dict = get_user_activate_degree_dict(all_click_df)
u2u_sim = usercf_sim(all_click_df, user_activate_degree_dict)

 32%|███▏      | 11371/35380 [34:19<1:12:27,  5.52it/s] 


KeyboardInterrupt: 

In [10]:
# itemcf sim召回
# 先进行itemcf召回, 为了召回评估，所以提取最后一次点击
metric_recall=False
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)

i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))
emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))

sim_item_topk = 20
recall_item_num = 10
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        item_topk_click, item_created_time_dict, emb_i2i_sim)

user_multi_recall_dict['itemcf_sim_itemcf_recall'] = user_recall_items_dict
pickle.dump(user_multi_recall_dict['itemcf_sim_itemcf_recall'], open(save_path + 'itemcf_recall_dict.pkl', 'wb'))

if metric_recall:
    # 召回效果评估
    metrics_recall(user_multi_recall_dict['itemcf_sim_itemcf_recall'], trn_last_click_df)

100%|██████████| 250000/250000 [1:40:36<00:00, 41.41it/s]  


In [11]:
# embedding sim召回
# 这里是为了召回评估，所以提取最后一次点击
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)
i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl','rb'))

sim_item_topk = 20
recall_item_num = 10

item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, sim_item_topk, 
                                                        recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim)
    
user_multi_recall_dict['embedding_sim_item_recall'] = user_recall_items_dict
pickle.dump(user_multi_recall_dict['embedding_sim_item_recall'], open(save_path + 'embedding_sim_item_recall.pkl', 'wb'))

if metric_recall:
    # 召回效果评估
    metrics_recall(user_multi_recall_dict['embedding_sim_item_recall'], trn_last_click_df, topk=recall_item_num)

100%|██████████| 250000/250000 [01:58<00:00, 2105.74it/s]


In [28]:
# 冷启动召回
# 先进行itemcf召回
trn_hist_click_df=all_click_df

user_recall_items_dict=collections.defaultdict(dict)
user_item_time_dict=get_user_item_time(trn_hist_click_df)
i2i_sim=pickle.load(open(save_path+'emb_i2i_sim.pkl','rb'))

sim_item_topk=150
recall_item_num=100 # 召回的文章数量

item_topk_click=get_item_topk_click(trn_hist_click_df,k=50)
for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user]=item_based_recommend(user,user_item_time_dict,i2i_sim,sim_item_topk,recall_item_num,item_topk_click,item_created_time_dict,emb_i2i_sim)
pickle.dump(user_recall_items_dict,open(save_path+'cold_start_items_raw_dict.pkl','wb'))

100%|██████████| 250000/250000 [02:24<00:00, 1729.99it/s]


In [12]:
all_click_df_=all_click_df.copy()
all_click_df_=all_click_df_.merge(item_info_df,how='left',on='click_article_id')
user_hist_item_typs_dict,user_hist_item_ids_dict,user_hist_item_words_dict,user_last_item_created_time_dict=get_user_hist_item_info_dict(all_click_df_)
click_article_ids_set = get_click_article_ids_set(all_click_df)
# 需要注意的是
# 这里使用了很多规则来筛选冷启动的文章，所以前面再召回的阶段就应该尽可能的多召回一些文章，否则很容易被删掉
cold_start_user_items_dict = cold_start_items(user_recall_items_dict, user_hist_item_typs_dict, user_hist_item_words_dict, \
                                              user_last_item_created_time_dict, item_type_dict, item_words_dict, \
                                              item_created_time_dict, click_article_ids_set, recall_item_num)

user_multi_recall_dict['cold_start_recall'] = cold_start_user_items_dict

100%|██████████| 250000/250000 [00:24<00:00, 10123.98it/s]


In [13]:
# 多路召回合并
# 对多路召回的权重给了一个相同的值
weight_dict = {'itemcf_sim_itemcf_recall': 1.0,
               'embedding_sim_item_recall': 1.0,
               'cold_start_recall': 1.0}

In [14]:
# 最终合并之后每个用户召回150个商品进行排序
final_recall_items_dict_rank = combine_recall_results(user_multi_recall_dict, weight_dict, topk=150)

多路召回合并...


  0%|          | 0/3 [00:00<?, ?it/s]

itemcf_sim_itemcf_recall...


 33%|███▎      | 1/3 [00:10<00:21, 10.99s/it]

embedding_sim_item_recall...


 67%|██████▋   | 2/3 [00:17<00:08,  8.08s/it]

cold_start_recall...


100%|██████████| 3/3 [00:18<00:00,  6.32s/it]


### 特征工程部分

#### 1. 分割数据集

In [8]:
click_trn, click_val, click_tst, val_ans = get_trn_val_tst_data(data_path, offline=False)

-- Mem. usage decreased to 23.34 Mb (69.4% reduction),time spend:0.00 min


In [33]:
click_tst.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,249999,160974,1506959142820,4,1,17,1,13,2
1,249999,160417,1506959172820,4,1,17,1,13,2
2,249998,160974,1506959056066,4,1,12,1,13,2
3,249998,202557,1506959086066,4,1,12,1,13,2
4,249997,183665,1506959088613,4,1,17,1,15,5


In [25]:
click_trn.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
119,199963,272143,1507029783117,4,1,17,1,10,2
120,199963,199198,1507029813117,4,1,17,1,10,2
250,199912,272934,1507033735624,4,1,17,1,20,4
251,199912,272143,1507033765624,4,1,17,1,20,4
2249,199203,156560,1507030851285,4,1,17,1,25,1


In [26]:
click_val.head()

user_id  click_article_id  click_timestamp  click_environment  \
user_id                                                                    
4031    0     4031             87215    1508171429515                  1   
4260    0     4260             39894    1508170043920                  4   
6989    0     6989            202381    1508157965153                  4   
        1     6989            166322    1508158085611                  4   
        2     6989            352121    1508172127990                  4   

           click_deviceGroup  click_os  click_country  click_region  \
user_id                                                               
4031    0                  3         2              1            25   
4260    0                  4        20              1             5   
6989    0                  1        17              1            21   
        1                  1        17              1            21   
        2                  1        17              1            21   

           click_referrer_type  
user_id                         
4031    0                    4  
4260    0                    2  
6989    0                    1  
        1                    1  
        2                    1

In [10]:
print("click_tst shape:",click_tst.shape)
print("click trn shape:",click_trn.shape)

click_tst shape: (518010, 9)
click trn shape: (1112623, 9)


In [35]:
val_ans.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
1090486,2546,57442,1508183897507,4,1,17,1,21,1
1084637,3083,4598,1508179446573,4,3,2,1,1,1
1061428,5245,331116,1508164941057,4,1,17,1,13,2
1056784,5688,181668,1508161269517,4,3,2,1,15,1
1023047,8947,16563,1508119971862,4,3,2,1,25,1


In [9]:
# 训练集的历史点击数据和最后一次点击
# click_trn_hist: 训练集的历史点击数据
# click_trn_last: 训练集的最后一次点击
click_trn_hist, click_trn_last = get_hist_and_last_click(click_trn)

if click_val is not None:
    click_val_hist, click_val_last = click_val, val_ans
else:
    click_val_hist, click_val_last = None, None
    
click_tst_hist = click_tst

In [59]:
click_tst_hist.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,249999,160974,1506959142820,4,1,17,1,13,2
1,249999,160417,1506959172820,4,1,17,1,13,2
2,249998,160974,1506959056066,4,1,12,1,13,2
3,249998,202557,1506959086066,4,1,12,1,13,2
4,249997,183665,1506959088613,4,1,17,1,15,5


#### 2. 负采样

In [10]:
# 读取召回列表
recall_list_dict = get_recall_list(save_path, single_recall_model='i2i_itemcf') # 这里只选择了单路召回的结果，也可以选择多路召回结果
# 将召回数据转换成df
recall_list_df = recall_dict_2_df(recall_list_dict)

100%|██████████| 250000/250000 [00:01<00:00, 216475.16it/s]


In [15]:
recall_list_df.head()

,user_id,sim_item,score
0,199999,276970,0.479507
1,199999,195545,0.368401
2,199999,158794,0.335175
3,199999,284096,0.326148
4,199999,124228,0.300447


In [11]:
# 如果有点击，label就会为1，如果没有点击，label就会为0
trn_user_item_label_df,val_user_item_label_df,tst_user_item_label_df=get_user_recall_item_label_df(click_val,click_trn_hist, click_val_hist, click_tst_hist,click_trn_last, click_val_last, recall_list_df)

pos_data_num: 0 neg_data_num: 2000000 pos/neg: 0.0


In [17]:
trn_user_item_label_df.head()

,user_id,sim_item,score,label
0,0,225446,0.364324,0.0
1,1,63783,0.154215,0.0
2,1,48401,0.156570,0.0
3,1,160807,0.176309,0.0
4,2,277107,0.216198,0.0


In [19]:
tst_user_item_label_df.head()

,user_id,sim_item,score,label
2000000,249999,234698,1.270511,-1
2000001,249999,95716,0.936343,-1
2000002,249999,336223,0.811552,-1
2000003,249999,59057,0.617284,-1
2000004,249999,300082,0.583236,-1


In [12]:
# 给训练验证数据打标签，并且负采样
trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df = get_user_recall_item_label_df(click_val,
                                                                                                       click_trn_hist, 
                                                                                                       click_val_hist, 
                                                                                                       click_tst_hist,
                                                                                                       click_trn_last, 
                                                                                                       click_val_last, 
                                                                                                       recall_list_df)


pos_data_num: 0 neg_data_num: 2000000 pos/neg: 0.0


#### 2. 召回数据转换为字典

In [21]:
tst_user_item_label_df.head()

,user_id,sim_item,score,label
2000000,249999,234698,1.270511,-1
2000001,249999,95716,0.936343,-1
2000002,249999,336223,0.811552,-1
2000003,249999,59057,0.617284,-1
2000004,249999,300082,0.583236,-1


In [22]:
trn_user_item_label_df.head()

,user_id,sim_item,score,label
0,0,119581,0.358026,0.0
1,0,225446,0.364324,0.0
2,1,63783,0.154215,0.0
3,1,48401,0.156570,0.0
4,1,30408,0.286269,0.0


In [23]:
tst_user_item_label_df.head()

,user_id,sim_item,score,label
2000000,249999,234698,1.270511,-1
2000001,249999,95716,0.936343,-1
2000002,249999,336223,0.811552,-1
2000003,249999,59057,0.617284,-1
2000004,249999,300082,0.583236,-1


In [13]:
trn_user_item_label_tuples=trn_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
trn_user_item_label_tuples_dict=dict(zip(trn_user_item_label_tuples['user_id'],trn_user_item_label_tuples[0]))

if val_user_item_label_df is not None:
    val_user_item_label_tuples=val_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
    val_user_item_label_tuples_dict=dict(zip(val_user_item_label_tuples['user_id'],val_user_item_label_tuples[0]))
else:
    val_user_item_label_tuples_dict=None


In [14]:
tst_user_item_label_tuples=tst_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()

In [15]:
tst_user_item_label_tuples_dict=dict(zip(tst_user_item_label_tuples['user_id'],tst_user_item_label_tuples[0]))

In [16]:
article_info_df = get_article_info_df()
all_click = pd.concat([click_trn, click_tst])
item_content_emb_dict, item_w2v_emb_dict = get_embedding(save_path, all_click)

-- Mem. usage decreased to  5.56 Mb (50.0% reduction),time spend:0.00 min


2024-10-30 22:40:45,465:INFO:collecting all words and their counts
2024-10-30 22:40:45,471:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-10-30 22:40:45,509:INFO:PROGRESS: at sentence #10000, processed 25727 words, keeping 3473 word types
2024-10-30 22:40:45,572:INFO:PROGRESS: at sentence #20000, processed 53883 words, keeping 5811 word types
2024-10-30 22:40:45,706:INFO:PROGRESS: at sentence #30000, processed 84881 words, keeping 7676 word types
2024-10-30 22:40:45,748:INFO:PROGRESS: at sentence #40000, processed 118390 words, keeping 9297 word types
2024-10-30 22:40:45,788:INFO:PROGRESS: at sentence #50000, processed 154179 words, keeping 10844 word types
2024-10-30 22:40:45,846:INFO:PROGRESS: at sentence #60000, processed 192350 words, keeping 12357 word types
2024-10-30 22:40:45,916:INFO:PROGRESS: at sentence #70000, processed 233685 words, keeping 13473 word types
2024-10-30 22:40:45,988:INFO:PROGRESS: at sentence #80000, processed 281335 words, keepin

In [28]:
article_info_df.head()

,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162


In [38]:
# 获取训练验证及测试数据中召回列文章相关特征
trn_user_item_feats_df = create_feature(trn_user_item_label_tuples_dict.keys(), trn_user_item_label_tuples_dict, \
                                            click_trn_hist, article_info_df, item_content_emb_dict)

100%|██████████| 200000/200000 [18:25<00:00, 180.96it/s] 


In [39]:
if val_user_item_label_tuples_dict is not None:
    val_user_item_feats_df = create_feature(val_user_item_label_tuples_dict.keys(), val_user_item_label_tuples_dict, \
                                                click_val_hist, article_info_df, item_content_emb_dict)
else:
    val_user_item_feats_df = None

In [40]:
tst_user_item_feats_df = create_feature(tst_user_item_label_tuples_dict.keys(), tst_user_item_label_tuples_dict, \
                                            click_tst_hist, article_info_df, item_content_emb_dict)

  0%|          | 0/50000 [00:00<?, ?it/s]

100%|██████████| 50000/50000 [23:41<00:00, 35.17it/s]  


In [41]:
trn_user_item_feats_df.to_csv(save_path+'trn_user_item_feats_df.csv',index=False)

if val_user_item_feats_df is not None:
    val_user_item_feats_df.to_csv(save_path+'val_user_item_feats_df.csv',index=False)

tst_user_item_feats_df.to_csv(save_path+'tst_user_item_feats_df.csv',index=False)

In [42]:
trn_user_item_feats_df.head()

,user_id,click_article_id,sim0,time_diff0,word_diff0,sim_max,sim_min,sim_sum,sim_mean,score,rank,label
0,0,119581.0,0.224301,73486000,104,0.224301,0.224301,0.224301,0.224301,0.358026,0,0.0
1,0,207747.0,0.163016,72344000,37,0.163016,0.163016,0.163016,0.163016,0.754372,1,0.0
2,1,63783.0,0.000427,293674000,59,0.000427,0.000427,0.000427,0.000427,0.154215,0,0.0
3,1,63760.0,-0.039672,466361000,84,-0.039672,-0.039672,-0.039672,-0.039672,0.208283,1,0.0
4,2,211442.0,0.647237,10549000,35,0.647237,0.647237,0.647237,0.647237,0.370236,0,0.0


In [43]:
tst_user_item_feats_df.head()

,user_id,click_article_id,sim0,time_diff0,word_diff0,sim_max,sim_min,sim_sum,sim_mean,score,rank,label
0,200000,194935.0,0.819129,2580000,15,0.819129,0.819129,0.819129,0.819129,0.714454,0,-1.0
1,200000,237870.0,0.323067,5706464000,26,0.323067,0.323067,0.323067,0.323067,0.652691,1,-1.0
2,200000,195645.0,0.708401,3010000,26,0.708401,0.708401,0.708401,0.708401,0.393158,2,-1.0
3,200000,50573.0,0.038526,8576289000,18,0.038526,0.038526,0.038526,0.038526,0.379683,3,-1.0
4,200000,16132.0,0.502068,654772000,51,0.502068,0.502068,0.502068,0.502068,0.343850,4,-1.0


### 用户相关特征

In [44]:
# 读取文章特征
articles=pd.read_csv(data_path+'articles.csv')
# articles=reduce_mem(articles)

# 日志数据
if click_val is not None:
    all_data=pd.concat([click_trn,click_val])
all_data=pd.concat([click_trn,click_tst])
# all_data=reduce_mem(all_data)

# 拼上文章信息
all_data=all_data.merge(articles,left_on='click_article_id',right_on='article_id')

#### 用户活跃度特征

In [45]:
user_act_fea=active_level(all_data,['user_id','click_article_id','click_timestamp'])

#### 文章热度特征

In [46]:
article_hot_fea=hot_level(all_data,['user_id','click_article_id','click_timestamp'])

#### 用户的设备习惯

In [47]:
# 设备特征
device_cols = ['user_id', 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']
user_device_info = device_fea(all_data, device_cols)

#### 用户的时间习惯

In [48]:
user_time_hob_cols=['user_id','click_timestamp','created_at_ts']
user_time_hob_info=user_time_hob_fea(all_data,user_time_hob_cols)

#### 用户的主题爱好

In [49]:
all_data.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type,article_id,category_id,created_at_ts,words_count
0,199999,160417,1507029570190,4,1,17,1,13,1,160417,281,1506942089000,173
1,199991,160417,1507029620497,4,1,17,1,2,2,160417,281,1506942089000,173
2,199989,160417,1507029634489,4,1,17,1,10,1,160417,281,1506942089000,173
3,199973,160417,1507029621819,4,1,17,1,16,2,160417,281,1506942089000,173
4,199968,160417,1507029586033,4,1,17,1,25,1,160417,281,1506942089000,173


In [50]:
user_category_hob_cols=['user_id','category_id']
user_category_hob_info=all_data[user_category_hob_cols]
user_category_hob_info=user_category_hob_info.groupby('user_id').agg({'category_id':list}).reset_index()

In [51]:
user_cat_hob_info=pd.DataFrame()
user_cat_hob_info['user_id']=user_category_hob_info['user_id']
user_cat_hob_info['cate_list']=user_category_hob_info['category_id']


#### 用户的字数偏好特征

In [52]:
user_wcou_info=all_data.groupby('user_id')['words_count'].agg('mean').reset_index()
user_wcou_info.rename(columns={'words_count':'words_hbo'},inplace=True)

#### 用户的信息特征合并保存

In [53]:
# 所有表进行合并
user_info=pd.merge(user_act_fea,user_device_info,on='user_id')
user_info=user_info.merge(user_time_hob_info,on='user_id')
user_info=user_info.merge(user_cat_hob_info,on='user_id')
user_info=user_info.merge(user_wcou_info,on='user_id')

In [54]:
user_info.to_csv(save_path+'user_info.csv',index=False)

#### 用户特征直接读入

In [55]:
# 读取用户信息
user_info=pd.read_csv(save_path+'user_info.csv')

In [56]:
if os.path.exists(save_path+'trn_user_item_feats_df.csv'):
    trn_user_item_feats_df=pd.read_csv(save_path+'trn_user_item_feats_df.csv')

if os.path.exists(save_path+'tst_user_item_feats_df.csv'):
    tst_user_item_feats_df=pd.read_csv(save_path+'tst_user_item_feats_df.csv')

if os.path.exists(save_path+'val_user_item_feats_df.csv'):
    val_user_item_feats_df=pd.read_csv(save_path+'val_user_item_feats_df.csv')
else:
    val_user_item_feats_df=None

In [57]:
# 拼上用户特征
trn_user_item_feats_df=trn_user_item_feats_df.merge(user_info,on='user_id',how='left')

if val_user_item_feats_df is not None:
    val_user_item_feats_df=val_user_item_feats_df.merge(user_info,on='user_id',how='left')
else:
    val_user_item_feats_df=None

tst_user_item_feats_df=tst_user_item_feats_df.merge(user_info,on='user_id',how='left')

#### 文章的特征直接读入

In [58]:
articles=pd.read_csv(data_path+'articles.csv')

In [59]:
# 拼上文章特征
trn_user_item_feats_df=trn_user_item_feats_df.merge(articles,left_on='click_article_id',right_on='article_id')

if val_user_item_feats_df is not None:
    val_user_item_feats_df=val_user_item_feats_df.merge(articles,left_on='click_article_id',right_on='article_id')
else:
    val_user_item_feats_df=None

tst_user_item_feats_df=tst_user_item_feats_df.merge(articles,left_on='click_article_id',right_on='article_id')

#### 召回文章的主题是否在用户的爱好里面

In [60]:
trn_user_item_feats_df.columns

Index(['user_id', 'click_article_id', 'sim0', 'time_diff0', 'word_diff0',
       'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'score', 'rank', 'label',
       'click_size', 'time_diff_mean', 'active_level', 'click_environment',
       'click_deviceGroup', 'click_os', 'click_country', 'click_region',
       'click_referrer_type', 'user_time_hob1', 'user_time_hob2', 'cate_list',
       'words_hbo', 'article_id', 'category_id', 'created_at_ts',
       'words_count'],
      dtype='object')

In [62]:
trn_user_item_feats_df['is_cat_hab']=trn_user_item_feats_df.apply(lambda x: 1 if x.category_id in set(x.cate_list) else 0, axis=1)
if val_user_item_feats_df is not None:
    val_user_item_feats_df['is_cat_hab']=val_user_item_feats_df.apply(lambda x: 1 if x.category_id in set(x.cate_list) else 0, axis=1)
else:
    val_user_item_feats_df=None

tst_user_item_feats_df['is_cat_hab']=tst_user_item_feats_df.apply(lambda x: 1 if x.category_id in set(x.cate_list) else 0, axis=1)

In [63]:
# 线下验证
del trn_user_item_feats_df['cate_list']

if val_user_item_feats_df is not None:
    del val_user_item_feats_df['cate_list']
else:
    val_user_item_feats_df=None

del tst_user_item_feats_df['cate_list']

del trn_user_item_feats_df['article_id']

if val_user_item_feats_df is not None:
    del val_user_item_feats_df['article_id']
else:
    val_user_item_feats_df=None

del tst_user_item_feats_df['article_id']

#### 保存特征

In [64]:
# 训练验证特征
trn_user_item_feats_df.to_csv(save_path+'trn_user_item_feats_df.csv',index=False)
if val_user_item_feats_df is not None:
    val_user_item_feats_df.to_csv(save_path+'val_user_item_feats_df.csv',index=False)
tst_user_item_feats_df.to_csv(save_path+'tst_user_item_feats_df.csv',index=False)

### 排序与模型融合

#### 读取排序特征

In [6]:
trn_user_item_feats_df=pd.read_csv(save_path+'trn_user_item_feats_df.csv')
trn_user_item_feats_df['click_article_id']=trn_user_item_feats_df['click_article_id'].astype(int)

offline=None
if offline:
    val_user_item_feats_df=pd.read_csv(save_path+'val_user_item_feats_df.csv')
    val_user_item_feats_df['click_article_id']=val_user_item_feats_df['click_article_id'].astype(int)
else:
    val_user_item_feats_df=None

tst_user_item_feats_df=pd.read_csv(save_path+'tst_user_item_feats_df.csv')
tst_user_item_feats_df['click_article_id']=tst_user_item_feats_df['click_article_id'].astype(int)

# 做特征的时候为了方便，给测试集也打上了一个无效的标签
del tst_user_item_feats_df['label']

#### 返回排序后的结果

##### LGB排序模型

In [7]:
# 防止中间出错之后重新读取数据
trn_user_item_feats_df_rank_model = trn_user_item_feats_df.copy()

if offline:
    val_user_item_feats_df_rank_model = val_user_item_feats_df.copy()
    
tst_user_item_feats_df_rank_model = tst_user_item_feats_df.copy()

In [8]:
# 定义特征列
lgb_cols = ['sim0', 'time_diff0', 'word_diff0','sim_max', 'sim_min', 'sim_sum', 
            'sim_mean', 'score','click_size', 'time_diff_mean', 'active_level',
            'click_environment','click_deviceGroup', 'click_os', 'click_country', 
            'click_region','click_referrer_type', 'user_time_hob1', 'user_time_hob2',
            'words_hbo', 'category_id', 'created_at_ts','words_count']

In [9]:
# 排序模型分组
trn_user_item_feats_df_rank_model.sort_values(by=['user_id'],inplace=True)
g_train=trn_user_item_feats_df_rank_model.groupby(['user_id'],as_index=False).count()['label'].values

if offline:
    val_user_item_feats_df_rank_model.sort_values(by=['user_id'],inplace=True)
    g_val=val_user_item_feats_df_rank_model.groupby(['user_id'],as_index=False).count()['label'].values

In [10]:
# 排序模型定义
lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)  

In [11]:
# 排序模型训练
if offline:
    lgb_ranker.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'], group=g_train,
                eval_set=[(val_user_item_feats_df_rank_model[lgb_cols], val_user_item_feats_df_rank_model['label'])], 
                eval_group= [g_val], eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ], early_stopping_rounds=50, )
else:
    lgb_ranker.fit(trn_user_item_feats_df[lgb_cols], trn_user_item_feats_df['label'], group=g_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011234 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4136
[LightGBM] [Info] Number of data points in the train set: 224230, number of used features: 23
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped train

In [12]:
# 模型预测
tst_user_item_feats_df['pred_score']=lgb_ranker.predict(tst_user_item_feats_df[lgb_cols],num_iteration=lgb_ranker.best_iteration_)

# 将这里的排序结果保存一份，用于后面的模型融合
tst_user_item_feats_df[['user_id','click_article_id','pred_score']].to_csv(save_path+'lgb_ranker_score.csv',index=False)

In [15]:
# 预测结果重新排序，及生成提交结果
rank_results=tst_user_item_feats_df[['user_id','click_article_id','pred_score']]
rank_results['click_article_id']=rank_results['click_article_id'].astype(int)
submit(rank_results,topk=5,model_name='lgb_ranker')

In [16]:
# 五折交叉验证，这里的五折交叉是以用户为目标进行五折划分
#  这一部分与前面的单独训练和验证是分开的
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_rank_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id','label']]
sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

# 五折交叉验证，并将中间结果保存用于staking
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    
    # 训练集与验证集的用户分组
    train_idx.sort_values(by=['user_id'], inplace=True)
    g_train = train_idx.groupby(['user_id'], as_index=False).count()["label"].values
    
    valid_idx.sort_values(by=['user_id'], inplace=True)
    g_val = valid_idx.groupby(['user_id'], as_index=False).count()["label"].values
    
    # 定义模型
    lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16)  
    # 训练模型
    lgb_ranker.fit(train_idx[lgb_cols], train_idx['label'], group=g_train,
                   eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], eval_group= [g_val], 
                   eval_at=[1, 2, 3, 4, 5], eval_metric=['ndcg', ])
    
    # 预测验证集结果
    valid_idx['pred_score'] = lgb_ranker.predict(valid_idx[lgb_cols], num_iteration=lgb_ranker.best_iteration_)
    
    # 对输出结果进行归一化
    valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))
    
    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 将验证集的预测结果放到一个列表中，后面进行拼接
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    
    # 如果是线上测试，需要计算每次交叉验证的结果相加，最后求平均
    if not offline:
        sub_preds += lgb_ranker.predict(tst_user_item_feats_df_rank_model[lgb_cols], lgb_ranker.best_iteration_)
    
score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
# 保存训练集交叉验证产生的新特征
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_ranker_feats.csv', index=False)
    
# 测试集的预测结果，多次交叉验证求平均,将预测的score和对应的rank特征保存，可以用于后面的staking，这里还可以构造其他更多的特征
tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# 保存测试集交叉验证的新特征
tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_ranker_feats.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008871 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4132
[LightGBM] [Info] Number of data points in the train set: 179462, number of used features: 23
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped train

In [17]:
# 预测结果重新排序, 及生成提交结果
# 单模型生成提交结果
rank_results = tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
submit(rank_results, topk=5, model_name='lgb_ranker')

#### LGB分类模型

In [19]:
# 模型及参数的定义
lgb_Classfication = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=500, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16, verbose=10) 

In [20]:
# 模型训练
if offline:
    lgb_Classfication.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'],
                    eval_set=[(val_user_item_feats_df_rank_model[lgb_cols], val_user_item_feats_df_rank_model['label'])], 
                    eval_metric=['auc', ],early_stopping_rounds=50, )
else:
    lgb_Classfication.fit(trn_user_item_feats_df_rank_model[lgb_cols], trn_user_item_feats_df_rank_model['label'])

[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 0, number of negative: 224230
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.042934
[LightGBM] [Debug] init for col-wise cost 0.000016 seconds, init for row-wise cost 0.030250 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4138
[LightGBM] [Info] Number of data points in the train set: 224230, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000000 -> initscore=-34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Debug] Re-bagging, using 157165 data to train
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 156722 data to train
[LightGBM] [Warning] Stopped training because there are no more 

In [21]:
# 模型预测
tst_user_item_feats_df['pred_score'] = lgb_Classfication.predict_proba(tst_user_item_feats_df[lgb_cols])[:,1]

# 将这里的排序结果保存一份，用户后面的模型融合
tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']].to_csv(save_path + 'lgb_cls_score.csv', index=False)

In [22]:
# 预测结果重新排序, 及生成提交结果
rank_results = tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
submit(rank_results, topk=5, model_name='lgb_cls')

In [24]:
# 五折交叉验证，这里的五折交叉是以用户为目标进行五折划分
#  这一部分与前面的单独训练和验证是分开的
def get_kfold_users(trn_df, n=5):
    user_ids = trn_df['user_id'].unique()
    user_set = [user_ids[i::n] for i in range(n)]
    return user_set

k_fold = 5
trn_df = trn_user_item_feats_df_rank_model
user_set = get_kfold_users(trn_df, n=k_fold)

score_list = []
score_df = trn_df[['user_id', 'click_article_id', 'label']]
sub_preds = np.zeros(tst_user_item_feats_df_rank_model.shape[0])

# 五折交叉验证，并将中间结果保存用于staking
for n_fold, valid_user in enumerate(user_set):
    train_idx = trn_df[~trn_df['user_id'].isin(valid_user)] # add slide user
    valid_idx = trn_df[trn_df['user_id'].isin(valid_user)]
    
    # 模型及参数的定义
    lgb_Classfication = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
                            max_depth=-1, n_estimators=100, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2018, n_jobs= 16, verbose=10)  
    # 训练模型
    lgb_Classfication.fit(train_idx[lgb_cols], train_idx['label'],eval_set=[(valid_idx[lgb_cols], valid_idx['label'])], 
                          eval_metric=['auc', ])
    
    # 预测验证集结果
    valid_idx['pred_score'] = lgb_Classfication.predict_proba(valid_idx[lgb_cols], 
                                                              num_iteration=lgb_Classfication.best_iteration_)[:,1]
    
    # 对输出结果进行归一化 分类模型输出的值本身就是一个概率值不需要进行归一化
    # valid_idx['pred_score'] = valid_idx[['pred_score']].transform(lambda x: norm_sim(x))
    
    valid_idx.sort_values(by=['user_id', 'pred_score'])
    valid_idx['pred_rank'] = valid_idx.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 将验证集的预测结果放到一个列表中，后面进行拼接
    score_list.append(valid_idx[['user_id', 'click_article_id', 'pred_score', 'pred_rank']])
    
    # 如果是线上测试，需要计算每次交叉验证的结果相加，最后求平均
    if not offline:
        sub_preds += lgb_Classfication.predict_proba(tst_user_item_feats_df_rank_model[lgb_cols], 
                                                     num_iteration=lgb_Classfication.best_iteration_)[:,1]
    
score_df_ = pd.concat(score_list, axis=0)
score_df = score_df.merge(score_df_, how='left', on=['user_id', 'click_article_id'])
# 保存训练集交叉验证产生的新特征
score_df[['user_id', 'click_article_id', 'pred_score', 'pred_rank', 'label']].to_csv(save_path + 'trn_lgb_cls_feats.csv', index=False)
    
# 测试集的预测结果，多次交叉验证求平均,将预测的score和对应的rank特征保存，可以用于后面的staking，这里还可以构造其他更多的特征
tst_user_item_feats_df_rank_model['pred_score'] = sub_preds / k_fold
tst_user_item_feats_df_rank_model['pred_score'] = tst_user_item_feats_df_rank_model['pred_score'].transform(lambda x: norm_sim(x))
tst_user_item_feats_df_rank_model.sort_values(by=['user_id', 'pred_score'])
tst_user_item_feats_df_rank_model['pred_rank'] = tst_user_item_feats_df_rank_model.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

# 保存测试集交叉验证的新特征
tst_user_item_feats_df_rank_model[['user_id', 'click_article_id', 'pred_score', 'pred_rank']].to_csv(save_path + 'tst_lgb_cls_feats.csv', index=False)

[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 0, number of negative: 179462
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.042919
[LightGBM] [Debug] init for col-wise cost 0.000017 seconds, init for row-wise cost 0.033084 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4132
[LightGBM] [Info] Number of data points in the train set: 179462, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000000 -> initscore=-34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Debug] Re-bagging, using 125726 data to train
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Debug] Re-bagging, using 125521 data to train
[LightGBM] [Warning] Stopped training because there are no more 

In [26]:
# 预测结果重新排序, 及生成提交结果
rank_results = tst_user_item_feats_df[['user_id', 'click_article_id', 'pred_score']]
rank_results['click_article_id'] = rank_results['click_article_id'].astype(int)
submit(rank_results, topk=5, model_name='lgb_cls')

### DIN模型

In [27]:
if offline:
    all_data=pd.read_csv(data_path+'train_click_log.csv')
else:
    trn_data=pd.read_csv(data_path+'train_click_log.csv')
    tst_data=pd.read_csv(data_path+'testA_click_log.csv')
    all_data=pd.concat([trn_data,tst_data])

In [29]:
hist_click=all_data[['user_id','click_article_id']].groupby('user_id').agg({list}).reset_index()
his_behavior_df = pd.DataFrame()
his_behavior_df['user_id'] = hist_click['user_id']
his_behavior_df['hist_click_article_id'] = hist_click['click_article_id']

In [30]:
trn_user_item_feats_df_din_model=trn_user_item_feats_df.copy()

if offline:
    val_user_item_feats_df_din_model=val_user_item_feats_df.copy()
else:
    val_user_item_feats_df_din_model=None

tst_user_item_feats_df_din_model=tst_user_item_feats_df.copy()

In [31]:
trn_user_item_feats_df_din_model = trn_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')

if offline:
    val_user_item_feats_df_din_model = val_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')
else:
    val_user_item_feats_df_din_model = None

tst_user_item_feats_df_din_model = tst_user_item_feats_df_din_model.merge(his_behavior_df, on='user_id')

In [32]:
# 把特征分开
sparse_fea=['user_id','click_article_id','category_id','click_environment','click_deviceGroup',
            'click_os','click_country','click_region','click_referrer_type','is_cat_hab']
behavior_fea=['click_article_id']
dense_fea = ['sim0', 'time_diff0', 'word_diff0', 'sim_max', 'sim_min', 'sim_sum', 'sim_mean', 'score',
             'rank','click_size','time_diff_mean','active_level','user_time_hob1','user_time_hob2',
             'words_hbo','words_count']
hist_behavior_fea = ['hist_click_article_id']

In [37]:
tst_user_item_feats_df_din_model.head()

,user_id,click_article_id,sim0,time_diff0,word_diff0,sim_max,sim_min,sim_sum,sim_mean,score,...,click_referrer_type,user_time_hob1,user_time_hob2,words_hbo,category_id,created_at_ts,words_count,is_cat_hab,pred_score,hist_click_article_id
0,200000,194935,0.819129,2580000,15,0.819129,0.819129,0.819129,0.819129,0.714454,...,1,0.076379,0.989986,200.333333,317,1507633570000,217,0,1.000000e-15,"[195839, 191971, 194300]"
1,200000,237870,0.323067,5706464000,26,0.323067,0.323067,0.323067,0.323067,0.652691,...,1,0.076379,0.989986,200.333333,375,1501929686000,228,0,1.000000e-15,"[195839, 191971, 194300]"
2,200000,195645,0.708401,3010000,26,0.708401,0.708401,0.708401,0.708401,0.393158,...,1,0.076379,0.989986,200.333333,317,1507639160000,176,0,1.000000e-15,"[195839, 191971, 194300]"
3,200000,50573,0.038526,8576289000,18,0.038526,0.038526,0.038526,0.038526,0.379683,...,1,0.076379,0.989986,200.333333,99,1499059861000,220,0,1.000000e-15,"[195839, 191971, 194300]"
4,200000,16132,0.502068,654772000,51,0.502068,0.502068,0.502068,0.502068,0.343850,...,1,0.076379,0.989986,200.333333,7,1506981378000,151,0,1.000000e-15,"[195839, 191971, 194300]"


In [95]:
tst_user_item_feats_df_din_model[dense_fea]

,sim0,time_diff0,word_diff0,sim_max,sim_min,sim_sum,sim_mean,score,rank,click_size,time_diff_mean,active_level,user_time_hob1,user_time_hob2,words_hbo,words_count
0,0.819129,2580000,15,0.819129,0.819129,0.819129,0.819129,0.714454,0,0.332622,0.501178,0.83380,0.076379,0.989986,200.333333,217
1,0.323067,5706464000,26,0.323067,0.323067,0.323067,0.323067,0.652691,1,0.332622,0.501178,0.83380,0.076379,0.989986,200.333333,228
2,0.708401,3010000,26,0.708401,0.708401,0.708401,0.708401,0.393158,2,0.332622,0.501178,0.83380,0.076379,0.989986,200.333333,176
3,0.038526,8576289000,18,0.038526,0.038526,0.038526,0.038526,0.379683,3,0.332622,0.501178,0.83380,0.076379,0.989986,200.333333,220
4,0.502068,654772000,51,0.502068,0.502068,0.502068,0.502068,0.343850,4,0.332622,0.501178,0.83380,0.076379,0.989986,200.333333,151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.115883,18574060000,65,0.115883,0.115883,0.115883,0.115883,1.228831,5,0.051621,0.054199,0.10582,0.062001,0.972165,217.368421,193
499996,0.253621,16328230000,18,0.253621,0.253621,0.253621,0.253621,1.216443,6,0.051621,0.054199,0.10582,0.062001,0.972165,217.368421,276
499997,-0.054219,1830719000,127,-0.054219,-0.054219,-0.054219,-0.054219,0.973949,7,0.051621,0.054199,0.10582,0.062001,0.972165,217.368421,131
499998,0.114561,64547637000,90,0.114561,0.114561,0.114561,0.114561,0.965390,8,0.051621,0.054199,0.10582,0.062001,0.972165,217.368421,168


In [33]:
# dense特征进行归一化，神经网络训练都需要将数值进行归一化处理
mm=MinMaxScaler()

for feat in dense_fea:
    trn_user_item_feats_df_din_model[feat]=mm.fit_transform(trn_user_item_feats_df_din_model[[feat]])

    if val_user_item_feats_df_din_model is not None:
        val_user_item_feats_df_din_model[feat]=mm.fit_transform(val_user_item_feats_df_din_model[[feat]])

    tst_user_item_feats_df_din_model[feat]=mm.fit_transform(tst_user_item_feats_df_din_model[[feat]])

In [34]:
# 准备训练数据
x_trn, dnn_feature_columns = get_din_feats_columns(trn_user_item_feats_df_din_model, dense_fea, 
                                               sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
y_trn = trn_user_item_feats_df_din_model['label'].values

if offline:
    # 准备验证数据
    x_val, dnn_feature_columns = get_din_feats_columns(val_user_item_feats_df_din_model, dense_fea, 
                                                   sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)
    y_val = val_user_item_feats_df_din_model['label'].values
    
dense_fea = [x for x in dense_fea if x != 'label']
x_tst, dnn_feature_columns = get_din_feats_columns(tst_user_item_feats_df_din_model, dense_fea, 
                                               sparse_fea, behavior_fea, hist_behavior_fea, max_len=50)

In [42]:
dnn_feature_columns

[SparseFeat(name='user_id', vocabulary_size=50001, embedding_dim=32, use_hash=False, vocabulary_path=None, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x142781670>, embedding_name='user_id', group_name='default_group', trainable=True),
 SparseFeat(name='click_article_id', vocabulary_size=15448, embedding_dim=32, use_hash=False, vocabulary_path=None, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x142781af0>, embedding_name='click_article_id', group_name='default_group', trainable=True),
 SparseFeat(name='category_id', vocabulary_size=242, embedding_dim=32, use_hash=False, vocabulary_path=None, dtype='int32', embeddings_initializer=<tensorflow.python.keras.initializers.initializers_v1.RandomNormal object at 0x142781850>, embedding_name='category_id', group_name='default_group', trainable=True),
 SparseFeat(name='click_environment', vocabulary_size=4

In [40]:
x_trn['user_id'].shape

(224230,)

In [35]:
# 建立模型
model = DIN(dnn_feature_columns, behavior_fea)

# 查看模型结构
model.summary()

# 模型编译
model.compile('adam', 'binary_crossentropy',metrics=['binary_crossentropy', tf.keras.metrics.AUC()])
 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
click_article_id (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
category_id (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
click_environment (InputLayer)  [(None, 1)]          0                                            
______________________________________________________________________________________________

In [53]:
x_trn

{'user_id': array([     0,      0,  90299, ..., 199581, 199630, 199990]),
 'click_article_id': array([119581, 207747, 119581, ..., 290327, 258026,  96999]),
 'category_id': array([247, 331, 247, ..., 421, 392, 209]),
 'click_environment': array([4, 4, 4, ..., 4, 4, 4]),
 'click_deviceGroup': array([1, 1, 3, ..., 1, 3, 1]),
 'click_os': array([17, 17,  2, ..., 17,  2, 17]),
 'click_country': array([1, 1, 1, ..., 1, 1, 1]),
 'click_region': array([25, 25,  6, ..., 25, 25,  7]),
 'click_referrer_type': array([2, 2, 1, ..., 5, 6, 1]),
 'is_cat_hab': array([0, 0, 0, ..., 0, 0, 0]),
 'sim0': array([0.42976765, 0.38471554, 0.41318171, ..., 0.4167342 , 0.47689167,
        0.53478639]),
 'time_diff0': array([0.00022973, 0.00022616, 0.00247717, ..., 0.00041442, 0.00374637,
        0.00099039]),
 'word_diff0': array([0.01600739, 0.00569494, 0.01277513, ..., 0.00554102, 0.0107742 ,
        0.00661844]),
 'sim_max': array([0.42976765, 0.38471554, 0.41318171, ..., 0.4167342 , 0.47689167,
        0.5

In [55]:
# 打印所有字段的最大值
print("Max user_id:", np.max(x_trn['user_id']))
print("Max click_article_id:", np.max(x_trn['click_article_id']))
print("Max category_id:", np.max(x_trn['category_id']))
print("Max click_environment:", np.max(x_trn['click_environment']))


Max user_id: 199999
Max click_article_id: 364046
Max category_id: 460
Max click_environment: 4


In [54]:
y_trn

array([0., 0., 0., ..., 0., 0., 0.])

In [39]:
# 模型训练
if offline:
    history = model.fit(x_trn, y_trn, verbose=1, epochs=10, validation_data=(x_val, y_val) , batch_size=256)
else:
    # 也可以使用上面的语句用自己采样出来的验证集
    # history = model.fit(x_trn, y_trn, verbose=1, epochs=3, validation_split=0.3, batch_size=256)
    history = model.fit(x_trn, y_trn, verbose=1, epochs=2, batch_size=256)

Epoch 1/2


2024-11-02 13:32:44.979376: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices[0,0] = 4 is not in [0, 4)
	 [[{{node model/sparse_emb_click_environment/embedding_lookup}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'model/sparse_emb_click_environment/embedding_lookup' defined at (most recent call last):
    File "/Users/linjiaxi/Anaconda/anaconda3/envs/news_recommendation_env/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/linjiaxi/Anaconda/anaconda3/envs/news_recommendation_env/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/linjiaxi/Anaconda/anaconda3/envs/news_recommendation_env/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/Users/linjiaxi/Anaconda/anaconda3/envs/news_recommendation_env/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/Users/linjiaxi/Anaconda/anaconda3/envs/news_recommendation_env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 739, in start
      

In [3]:
# 模型预测
tst_user_item_feats_df_din_model['pred_score']=model.predict(x_tst,verbose=1,batch_size=256)
tst_user_item_feats_df_din_model[['user_id','click_article_id','pred_score']].to_csv(save_path,'din_rank_score.csv',index=False)

NameError: name 'model' is not defined

In [ ]:
# 预测结果重新排序，以及生成提交结果
rank_results=tst_user_item_feats_df_din_model[['user_id','click_article_id','pred_score']]
submit(rank_results,topk=5,model_name='din')

#### DIEN模型

In [42]:
import pandas as pd
from keras_dien import DIEN
from keras_dien.utils import create_features, get_dien_input

ModuleNotFoundError: No module named 'keras_dien'

### 模型融合

#### 加权融合

In [ ]:
# 读取多个模型的排序结果文件
lgb_ranker = pd.read_csv(save_path + 'lgb_ranker_score.csv')
lgb_cls = pd.read_csv(save_path + 'lgb_cls_score.csv')
din_ranker = pd.read_csv(save_path + 'din_rank_score.csv')

# 这里也可以换成交叉验证输出的测试结果进行加权融合

In [ ]:
rank_model = {'lgb_ranker': lgb_ranker, 
              'lgb_cls': lgb_cls, 
              'din_ranker': din_ranker}

In [ ]:
get_ensemble_predict_topk(rank_model)

#### Stacking

In [ ]:
# 读取多个模型的交叉验证生成的结果文件
# 训练集
trn_lgb_ranker_feats = pd.read_csv(save_path + 'trn_lgb_ranker_feats.csv')
trn_lgb_cls_feats = pd.read_csv(save_path + 'trn_lgb_cls_feats.csv')
trn_din_cls_feats = pd.read_csv(save_path + 'trn_din_cls_feats.csv')

# 测试集
tst_lgb_ranker_feats = pd.read_csv(save_path + 'tst_lgb_ranker_feats.csv')
tst_lgb_cls_feats = pd.read_csv(save_path + 'tst_lgb_cls_feats.csv')
tst_din_cls_feats = pd.read_csv(save_path + 'tst_din_cls_feats.csv')

In [ ]:
# 将多个模型输出的特征进行拼接

finall_trn_ranker_feats = trn_lgb_ranker_feats[['user_id', 'click_article_id', 'label']]
finall_tst_ranker_feats = tst_lgb_ranker_feats[['user_id', 'click_article_id']]

for idx, trn_model in enumerate([trn_lgb_ranker_feats, trn_lgb_cls_feats, trn_din_cls_feats]):
    for feat in [ 'pred_score', 'pred_rank']:
        col_name = feat + '_' + str(idx)
        finall_trn_ranker_feats[col_name] = trn_model[feat]

for idx, tst_model in enumerate([tst_lgb_ranker_feats, tst_lgb_cls_feats, tst_din_cls_feats]):
    for feat in [ 'pred_score', 'pred_rank']:
        col_name = feat + '_' + str(idx)
        finall_tst_ranker_feats[col_name] = tst_model[feat]

In [ ]:
# 定义一个逻辑回归模型再次拟合交叉验证产生的特征对测试集进行预测
# 这里需要注意的是，在做交叉验证的时候可以构造多一些与输出预测值相关的特征，来丰富这里简单模型的特征
from sklearn.linear_model import LogisticRegression

feat_cols = ['pred_score_0', 'pred_rank_0', 'pred_score_1', 'pred_rank_1', 'pred_score_2', 'pred_rank_2']

trn_x = finall_trn_ranker_feats[feat_cols]
trn_y = finall_trn_ranker_feats['label']

tst_x = finall_tst_ranker_feats[feat_cols]

# 定义模型
lr = LogisticRegression()

# 模型训练
lr.fit(trn_x, trn_y)

# 模型预测
finall_tst_ranker_feats['pred_score'] = lr.predict_proba(tst_x)[:, 1]

In [ ]:
# 预测结果重新排序, 及生成提交结果
rank_results = finall_tst_ranker_feats[['user_id', 'click_article_id', 'pred_score']]
submit(rank_results, topk=5, model_name='ensumble_staking')